# Продвинутое машинное обучение: ДЗ 3
## *Латыпов Ильяс, группа DS-22*
## Часть 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:

    - подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
    - возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
    - расшифруйте их таким частотным методом.

In [1]:
import numpy as np
import re
import random
from nltk.util import ngrams
from collections import Counter

### Загрузка и обработка данных

In [2]:
PATH_ANNA_RU = '.\data\AnnaKarenina.txt'
PATH_WAR_RU = '.\data\WarAndPeace.txt'
PATH_WAR_EN = '.\data\WarAndPeaceEng.txt'

def load_text(path):
    with open(path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

text_anna_ru = load_text(PATH_ANNA_RU)
text_war_ru = load_text(PATH_WAR_RU)
text_war_en = load_text(PATH_WAR_EN)

print(f"Длинна текста AnnaKarenina.txt: {len(text_anna_ru)}")
print(f"Длинна текста WarAndPeace.txt: {len(text_war_ru)}")
print(f"Длинна текста WarAndPeaceEng.txt: {len(text_war_en)}")

Длинна текста AnnaKarenina.txt: 1813200
Длинна текста WarAndPeace.txt: 717873
Длинна текста WarAndPeaceEng.txt: 3226615


In [3]:
EN_TOKKENS = list('abcdefghijklmnopqrstuvwxyz ')
RU_TOKKENS = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

def prepare_text(text, tokkens):
    text = text.lower()
    text = text.replace("\n"," ").replace("."," ")
    if "s" in tokkens: #английский
        text = text.replace("chapter"," ").replace("iii"," ").replace("ii"," ").replace("xx"," ")
        text = text.replace(" i "," ").replace(" ii "," ").replace(" iii "," ").replace(" iv "," ").replace(" v "," ")
        text = text.replace(" vi "," ").replace(" vii "," ").replace(" viii "," ").replace(" ix "," ").replace(" x "," ")
        text = text.replace(" xi "," ").replace(" xii "," ").replace(" xiii "," ").replace(" xiv "," ").replace("xv "," ")
        text = text.replace(" xvi "," ").replace(" xvii "," ").replace(" xviii "," ").replace(" xix "," ").replace(" xx "," ")
    
    text = ''.join([char_ for char_ in text if char_ in tokkens])
    space = re.compile(r'\s+')
    return space.sub(' ', text)
    
text_anna_ru = prepare_text(text_anna_ru, RU_TOKKENS)
text_war_ru = prepare_text(text_war_ru, RU_TOKKENS)
text_war_en = prepare_text(text_war_en, EN_TOKKENS)
print("Примеры отрывков загруженных произведений:", end="\n\n")
print(text_anna_ru[1000:1305], end="\n\n")
print(text_war_ru[1000:1300], end="\n\n")
print(text_war_en[999:1305])

Примеры отрывков загруженных произведений:

 в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения время подтвердило эту высокую оценку из статей и книг на всех языках мира посвященных анне карениной можно составить целую библиотеку

 я вижу что я вас пугаю садитесь и рассказывайте так говорила в июле года известная анна павловна шерер фрейлина и приближенная императрицы марии феодоровны встречая важного и чиновного князя василия первого приехавшего на ее вечер анна павловна кашляла несколько дней у нее был грипп как она говорил

really believe he is antichristi will have nothing more to do with you and you are no longer my friend no longer my faithful slave as you call yourself but how do you do see have frightened yousit down and tell me all the news it was in july and the speaker was the wellknown anna pavlovna scherer maid of 


In [4]:
def get_ngrams(text, n):
    n_grams = ngrams(text, n)
    return [''.join(grams) for grams in n_grams]

ru_char_order = Counter(get_ngrams(text_anna_ru + text_war_ru,1)).most_common()
print("Частоты для русского языка:\n", ru_char_order)

Частоты для русского языка:
 [(' ', 385247), ('о', 223691), ('е', 166169), ('а', 162313), ('н', 133258), ('и', 129712), ('т', 115258), ('с', 103252), ('л', 98191), ('в', 91386), ('р', 80859), ('к', 67788), ('д', 58019), ('м', 56467), ('у', 53583), ('п', 47940), ('я', 42921), ('ь', 38351), ('г', 36870), ('ы', 36447), ('б', 34028), ('з', 32723), ('ч', 31207), ('ж', 21480), ('й', 21072), ('ш', 17158), ('х', 15586), ('ю', 12307), ('э', 6648), ('ц', 6172), ('щ', 5568), ('ф', 2990), ('ъ', 695), ('ё', 462)]


In [5]:
en_char_order = Counter(get_ngrams(text_war_en,1)).most_common()
print("Частоты для английского языка:\n", en_char_order)

Частоты для английского языка:
 [(' ', 561009), ('e', 314501), ('t', 225674), ('a', 205074), ('o', 192879), ('n', 184173), ('i', 169985), ('h', 166672), ('s', 162891), ('r', 147696), ('d', 118290), ('l', 96527), ('u', 65434), ('m', 61646), ('c', 60891), ('w', 59207), ('f', 54896), ('g', 51326), ('y', 46265), ('p', 44801), ('b', 34658), ('v', 26713), ('k', 20431), ('x', 3721), ('j', 2574), ('z', 2388), ('q', 2330)]


### Шифровка и расшифровка текста на **русском** языке частотным методом (для 1 символа)

In [6]:
RU_TOKKENS_SHUFFLE = RU_TOKKENS.copy()
random.shuffle(RU_TOKKENS_SHUFFLE)
dict_encoder_ru = dict(zip(RU_TOKKENS, RU_TOKKENS_SHUFFLE))
print("Словарь для шифровки: ")
dict_encoder_ru

Словарь для шифровки: 


{'а': 'о',
 'б': 'н',
 'в': 'ь',
 'г': 'ж',
 'д': 'ц',
 'е': 'г',
 'ё': 'ъ',
 'ж': 'м',
 'з': 'х',
 'и': 'ё',
 'й': 'л',
 'к': 'у',
 'л': 'е',
 'м': 'б',
 'н': ' ',
 'о': 'ш',
 'п': 'в',
 'р': 'ч',
 'с': 'и',
 'т': 'а',
 'у': 'к',
 'ф': 'п',
 'х': 'ф',
 'ц': 'д',
 'ч': 'щ',
 'ш': 'р',
 'щ': 'э',
 'ъ': 'с',
 'ы': 'я',
 'ь': 'й',
 'э': 'т',
 'ю': 'ю',
 'я': 'з',
 ' ': 'ы'}

In [7]:
text_for_encode = text_anna_ru[1001:3959]
print("Отрывок текста для шифровки:\n", text_for_encode)

Отрывок текста для шифровки:
 в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения время подтвердило эту высокую оценку из статей и книг на всех языках мира посвященных анне карениной можно составить целую библиотеку я без колебаний назвал анну каренину величайшим социальным романом во всей мировой литературе писал томас манн значение романа толстого состоит не в эстетической ценности отдельных картин а в художественной завершенности целого войну и мир толстой называл книгой о прошлом в начале года он просил редактора журнала русский вестник м н каткова в оглавлении и даже в объявлении не называть его сочинение романом для меня это очень важно и потому очень прошу вас об этом толстой мог бы обосновать свое определение жанра книга ссылкой на гегеля которого он внимательно перечитывал в годы работы над войной и миром гегель называл книгой эпические произведения связанные 

In [8]:
text_encoded = "".join(map(lambda x: dict_encoder_ru[x], text_for_encode))
print("Зашифрованный отрывок текста:\n", text_encoded)

Зашифрованный отрывок текста:
 ьы шьшбычшбо гыашеиашжшышжчшб кюывиёфшешжёщгиукюычохчоншаукыцкрёыщгешьгщгиушлыиачор кюыжекнё кыёыиёекыёыжеоь шгы гняьоеялыцшигегыкы оиычгоеёхбыфкцшмгиаьг  шжшыёхшнчомг ёзыьчгбзывшцаьгчцёешытакыьяишукюышдг укыёхыиаоаглыёыу ёжы оыьигфызхяуофыбёчоывшиьзэг  яфыо  гыуочг ё шлыбшм шыишиаоьёайыдгекюынёнеёшагукызынгхыушегно ёлы охьоеыо  кыуочг ё кыьгеёщолрёбыишдёоей ябычшбо шбыьшыьиглыбёчшьшлыеёагчоакчгывёиоеыашбоиыбо  ых ощг ёгычшбо оыашеиашжшыишиашёаы гыьытиагаёщгиушлыдг  шиаёышацгей яфыуочаё ыоыьыфкцшмгиаьг  шлыхоьгчрг  шиаёыдгешжшыьшл кыёыбёчыашеиашлы охяьоеыу ёжшлышывчшрешбыьы ощоегыжшцоыш ывчшиёеычгцоуашчоымкч оеоычкииуёлыьгиа ёуыбы ыуоаушьоыьышжеоьег ёёыёыцомгыьышнсзьег ёёы гы охяьоайыгжшыишщё г ёгычшбо шбыцезыбг зыташышщг йыьом шыёывшашбкышщг йывчшркыьоиышныташбыашеиашлыбшжыняышнши шьоайыиьшгышвчгцгег ёгымо чоыу ёжоыиияеушлы оыжгжгезыушашчшжшыш ыь ёбоагей шывгчгщёаяьоеыьыжшцяычоншаяы оцыьшл шлыёыбёчшбыжгжгейы охяьоеыу ёжшлытвёщгиуёгывчшёхьгцг ёзыиьзхо  яг

In [9]:
text_encoded_order = Counter(get_ngrams(text_encoded,1)).most_common()
print("Частоты зашифрованного отрывка текста:\n", text_encoded_order)

Частоты зашифрованного отрывка текста:
 [('ы', 437), ('ш', 316), (' ', 231), ('г', 207), ('ё', 189), ('о', 181), ('и', 146), ('а', 142), ('ч', 122), ('ь', 117), ('е', 95), ('б', 79), ('у', 68), ('к', 58), ('л', 57), ('я', 55), ('ц', 53), ('в', 50), ('х', 46), ('ж', 45), ('н', 43), ('з', 42), ('ф', 31), ('щ', 29), ('й', 29), ('м', 21), ('т', 19), ('р', 15), ('д', 12), ('э', 9), ('ю', 7), ('п', 5), ('с', 2)]


In [10]:
dict_decoder_from_text = dict(zip(map(lambda x: x[0], text_encoded_order), map(lambda x: x[0], ru_char_order)))
text_decoded = "".join(map(lambda x: dict_decoder_from_text[x], text_encoded))
print("Расшифрованный отрывок текста:\n", text_decoded)

Расшифрованный отрывок текста:
 в еовок локиеа сортсоыо оылокемщ ьтнчороынжатдмщ лиглибосдм ямюн жароважатдоу тслиюемщ ырмбнем н тнрм н ыривеоа еабпвирпу яотара м еит лаирнгк чмяошатсваееоыо нгоблишаенз влакз ьоясвалянро хсм вптодмщ оэаедм нг тсисау н дены еи втач згпдич кнли ьотвзцаеепч иееа дилаенеоу кошео тотсивнсй эармщ бнбрносадм з баг дорабиену еигвир иеем дилаенем варнжиуюнк тоэнирйепк локиеок во втау кнловоу рнсалисмла ьнтир сокит киее геижаена локиеи сортсоыо тотсонс еа в хтсаснжатдоу эаееотсн осяарйепч дилсне и в чмяошатсваееоу гивалюаееотсн эароыо воуем н кнл сортсоу еигпвир деныоу о ьлоюрок в еижира ыояи ое ьлотнр лаяидсоли шмлеири лмттдну ватсенд к е дисдови в оыривраенн н яиша в объзвраенн еа еигпвисй аыо тожнеаена локиеок ярз каез хсо ожаей вишео н ьосокм ожаей ьлоюм вит об хсок сортсоу коы бп оботеовисй твоа оьлаяараена шиели деныи ттпрдоу еи ыаыарз досолоыо ое венкисарйео ьалажнспвир в ыояп либосп еия воуеоу н кнлок ыаыарй еигпвир деныоу хьнжатдна ьлонгваяаенз твзгиееп

In [11]:
def text_accuracy(text_true, text_pred):
    count_chars = 0
    len_ = min(len(text_true), len(text_pred))
    for idx in range(len_):
        if text_true[idx] == text_pred[idx]:
            count_chars += 1
    return count_chars/len_

print(f"Точность дешифровки для отрывка текста: {text_accuracy(text_for_encode, text_decoded) * 100:.2f}%")

Точность дешифровки для отрывка текста: 31.10%


### Шифровка и расшифровка текста на **английском** языке частотным методом (для 1 символа)

In [12]:
EN_TOKKENS_SHUFFLE = EN_TOKKENS.copy()
random.shuffle(EN_TOKKENS_SHUFFLE)
dict_encoder_en = dict(zip(EN_TOKKENS, EN_TOKKENS_SHUFFLE))
print("Словарь для шифровки: ")
dict_encoder_en

Словарь для шифровки: 


{'a': 'i',
 'b': 'd',
 'c': 'b',
 'd': 'o',
 'e': 'm',
 'f': 'x',
 'g': 'p',
 'h': 'u',
 'i': 's',
 'j': 'y',
 'k': 'h',
 'l': 't',
 'm': 'q',
 'n': 'f',
 'o': 'l',
 'p': 'n',
 'q': 'a',
 'r': ' ',
 's': 'g',
 't': 'c',
 'u': 'r',
 'v': 'w',
 'w': 'j',
 'x': 'v',
 'y': 'e',
 'z': 'z',
 ' ': 'k'}

In [13]:
text_for_encode = text_war_en[1030:4322]
print("Отрывок текста для шифровки:\n", text_for_encode)

Отрывок текста для шифровки:
 i will have nothing more to do with you and you are no longer my friend no longer my faithful slave as you call yourself but how do you do see have frightened yousit down and tell me all the news it was in july and the speaker was the wellknown anna pavlovna scherer maid of honor and favorite of the empress marya fedorovna with these words she greeted prince vasili kuragin a man of high rank and importance who was the first to arrive at her reception anna pavlovna had had a cough for some days she was as she said suffering from la grippe grippe being then a new word in st petersburg used only by the elite all her invitations without exception written in french and delivered by a scarletliveried footman that morning ran as follows if you have nothing better to do count or prince and if the prospect of spending an evening with a poor invalid is not too terrible shall be very charmed to see you tonight between and annette scherer heavens what a virulent attac

In [14]:
text_encoded = "".join(map(lambda x: dict_encoder_en[x], text_for_encode))
print("Зашифрованный отрывок текста:\n", text_encoded)

Зашифрованный отрывок текста:
 skjsttkuiwmkflcusfpkql mkclkolkjscukelrkifokelrki mkflktlfpm kqekx smfokflktlfpm kqekxiscuxrtkgtiwmkigkelrkbittkelr gmtxkdrckuljkolkelrkolkgmmkuiwmkx spucmfmokelrgsckoljfkifokcmttkqmkittkcumkfmjgksckjigksfkyrtekifokcumkgnmihm kjigkcumkjmtthfljfkiffikniwtlwfikgbum m kqisoklxkulfl kifokxiwl scmklxkcumkmqn mggkqi eikxmol lwfikjscukcumgmkjl ogkgumkp mmcmokn sfbmkwigstskhr ipsfkikqifklxkuspuk ifhkifoksqnl cifbmkjulkjigkcumkxs gckclki  swmkickum k mbmncslfkiffikniwtlwfikuiokuiokikblrpukxl kglqmkoiegkgumkjigkigkgumkgisokgrxxm sfpkx lqktikp snnmkp snnmkdmsfpkcumfkikfmjkjl oksfkgcknmcm gdr pkrgmoklftekdekcumkmtscmkittkum ksfwscicslfgkjsculrckmvbmncslfkj sccmfksfkx mfbukifokomtswm mokdekikgbi tmctswm smokxllcqifkcuickql fsfpk ifkigkxlttljgksxkelrkuiwmkflcusfpkdmccm kclkolkblrfckl kn sfbmkifoksxkcumkn lgnmbcklxkgnmfosfpkifkmwmfsfpkjscukiknll ksfwitsoksgkflckcllkcm  sdtmkguittkdmkwm ekbui qmokclkgmmkelrkclfspuckdmcjmmfkifokiffmccmkgbum m kumiwmfgkjuickikws rtmfckicci

In [15]:
text_encoded_order = Counter(get_ngrams(text_encoded,1)).most_common()
print("Частоты зашифрованного отрывка текста:\n", text_encoded_order)

Частоты зашифрованного отрывка текста:
 [('k', 591), ('m', 356), ('f', 231), ('i', 229), ('c', 219), ('s', 194), ('l', 194), ('g', 165), ('u', 156), (' ', 151), ('o', 132), ('t', 101), ('b', 81), ('r', 63), ('x', 63), ('n', 60), ('j', 59), ('e', 50), ('q', 45), ('w', 43), ('p', 42), ('d', 40), ('h', 20), ('v', 4), ('y', 2), ('z', 1)]


In [16]:
dict_decoder_from_text = dict(zip(map(lambda x: x[0], text_encoded_order), map(lambda x: x[0], en_char_order)))
text_decoded = "".join(map(lambda x: dict_decoder_from_text[x], text_encoded))
print("Расшифрованный отрывок текста:\n", text_decoded)

Расшифрованный отрывок текста:
 n fnll sape tiosntb yire oi di fnos gim atd gim are ti litber yg crnetd ti litber yg canoscml hlape ah gim uall gimrhelc vmo sif di gim di hee sape crnbsoeted gimhno dift atd oell ye all ose tefh no fah nt jmlg atd ose hweaker fah ose fellktift atta waplipta huserer yand ic sitir atd capirnoe ic ose eywrehh yarga cediripta fnos osehe firdh hse breeoed wrntue pahnln kmrabnt a yat ic snbs ratk atd nywiroatue fsi fah ose cnrho oi arrnpe ao ser reuewonit atta waplipta sad sad a uimbs cir hiye dagh hse fah ah hse hand hmccerntb criy la brnwwe brnwwe ventb oset a tef fird nt ho weoerhvmrb mhed itlg vg ose elnoe all ser ntpnoaonith fnosimo exuewonit frnooet nt cretus atd delnpered vg a huarleolnperned ciioyat osao yirtntb rat ah cillifh nc gim sape tiosntb veooer oi di uimto ir wrntue atd nc ose wrihweuo ic hwetdntb at epetntb fnos a wiir ntpalnd nh tio oii oerrnvle hsall ve perg usaryed oi hee gim oitnbso veofeet atd atteooe huserer seapeth fsao a pnrmleto aoo

In [17]:
print(f"Точность дешифровки для отрывка текста: {text_accuracy(text_for_encode, text_decoded) * 100:.2f}%")

Точность дешифровки для отрывка текста: 48.21%


### Выводы
  - Частотный подход только на одиночных символах к сожалению расшифровывает плохо, возможно не хватило длины зашифрованного отрывка текста

## Часть 2.

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

    - подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
    - проведите тестирование аналогично п.1, но при помощи биграмм

In [18]:
ru_char_order = Counter(get_ngrams(text_anna_ru + text_war_ru,2)).most_common()
print("Частоты биграмм для русского языка (первые 100):\n", ru_char_order[:100])

Частоты биграмм для русского языка (первые 100):
 [('о ', 53083), ('а ', 42142), ('и ', 42011), ('е ', 41730), (' с', 37090), (' н', 36857), (' в', 34614), (' п', 33496), ('то', 32646), (' о', 30491), ('я ', 25887), (' и', 25549), ('на', 24966), ('ст', 24325), ('ь ', 24219), (' к', 22321), ('не', 22125), ('ал', 21802), ('но', 21658), ('го', 19814), ('по', 19153), ('ко', 18402), ('он', 18290), ('ка', 17764), (' т', 17583), ('ов', 17438), ('ни', 17206), ('л ', 17162), ('й ', 17051), ('ла', 16879), ('ра', 16870), (' д', 16722), ('ен', 16543), ('м ', 16441), ('во', 16240), (' б', 15983), ('ро', 15967), ('у ', 15236), ('от', 15230), ('пр', 14196), ('ос', 14022), (' ч', 13848), ('ло', 13707), ('ть', 13543), ('ол', 13528), ('ел', 13448), ('ер', 13096), ('ор', 13081), ('в ', 13067), (' м', 13027), ('ли', 12895), ('ва', 12884), ('ре', 12626), ('за', 12415), ('н ', 12058), ('та', 11752), ('ск', 11460), ('те', 11307), (' е', 11255), ('ом', 11203), ('ле', 11127), ('ан', 11050), ('к ', 10982), ('ри

In [19]:
en_char_order = Counter(get_ngrams(text_war_en,2)).most_common()
print("Частоты биграмм для английского языка (первые 100):\n", en_char_order[:100])

Частоты биграмм для английского языка (первые 100):
 [('e ', 111106), (' t', 87536), ('d ', 75865), ('he', 75251), ('th', 73770), (' a', 69554), ('s ', 62857), ('t ', 58261), (' h', 49801), ('in', 48429), ('an', 45721), ('er', 43486), (' s', 40788), (' w', 40638), ('n ', 40545), ('re', 35670), ('nd', 34582), (' o', 34096), ('y ', 31379), ('r ', 30757), ('ed', 28702), ('at', 28179), ('on', 27731), (' i', 27406), ('ha', 27017), ('o ', 26908), ('en', 26584), ('hi', 25998), (' b', 25159), ('ng', 24992), ('to', 24751), ('ou', 23549), ('g ', 22136), ('is', 21988), ('it', 21463), (' f', 21368), ('as', 21176), ('es', 20486), (' c', 20457), ('or', 19801), (' m', 19526), ('f ', 19437), (' p', 18935), ('te', 18417), ('ar', 17955), ('st', 17846), ('se', 17569), (' d', 17244), ('nt', 17189), ('le', 17136), ('of', 16920), ('h ', 15429), ('me', 15180), ('a ', 15144), (' r', 15135), (' n', 15095), ('ne', 14843), ('ro', 14793), ('ve', 14585), ('al', 14235), ('ti', 14022), ('de', 13774), ('ho', 13726), 

### Шифровка и расшифровка текста на **русском** языке частотным методом (для биграмм)

In [20]:
RU_TOKKENS_BI = list(map(lambda x: x[0], ru_char_order))
RU_TOKKENS_BI_SHUFFLE = RU_TOKKENS_BI.copy()
random.shuffle(RU_TOKKENS_BI_SHUFFLE)
dict_encoder_ru = dict(zip(RU_TOKKENS_BI, RU_TOKKENS_BI_SHUFFLE))
print("Токкены биграмм для руссокого языка (первые 100):\n", RU_TOKKENS_BI[:100])

Токкены биграмм для руссокого языка (первые 100):
 ['о ', 'а ', 'и ', 'е ', ' с', ' н', ' в', ' п', 'то', ' о', 'я ', ' и', 'на', 'ст', 'ь ', ' к', 'не', 'ал', 'но', 'го', 'по', 'ко', 'он', 'ка', ' т', 'ов', 'ни', 'л ', 'й ', 'ла', 'ра', ' д', 'ен', 'м ', 'во', ' б', 'ро', 'у ', 'от', 'пр', 'ос', ' ч', 'ло', 'ть', 'ол', 'ел', 'ер', 'ор', 'в ', ' м', 'ли', 'ва', 'ре', 'за', 'н ', 'та', 'ск', 'те', ' е', 'ом', 'ле', 'ан', 'к ', 'ри', 'ог', 'ил', 'т ', 'де', 'ат', 'ет', 'од', 'ы ', 'ве', 'ак', 'ви', 'да', ' з', 'аз', 'чт', ' у', ' р', 'бы', 'же', ' г', 'ем', 'се', 'ин', 'ес', 'ти', 'ит', 'до', 'ас', 'ег', 'ль', 'ю ', ' л', 'об', 'вс', 'че', 'мо']


In [21]:
text_for_encode = text_anna_ru[1001:3959]
print("Отрывок текста для шифровки:\n", text_for_encode)

Отрывок текста для шифровки:
 в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения время подтвердило эту высокую оценку из статей и книг на всех языках мира посвященных анне карениной можно составить целую библиотеку я без колебаний назвал анну каренину величайшим социальным романом во всей мировой литературе писал томас манн значение романа толстого состоит не в эстетической ценности отдельных картин а в художественной завершенности целого войну и мир толстой называл книгой о прошлом в начале года он просил редактора журнала русский вестник м н каткова в оглавлении и даже в объявлении не называть его сочинение романом для меня это очень важно и потому очень прошу вас об этом толстой мог бы обосновать свое определение жанра книга ссылкой на гегеля которого он внимательно перечитывал в годы работы над войной и миром гегель называл книгой эпические произведения связанные 

In [22]:
def get_pair_char(text):
    tokkens = []
    for i in range(len(text)//2):
        tokkens.append(text[i*2:i*2+2])
    return tokkens

text_encoded = "".join(map(lambda x: dict_encoder_ru[x], get_pair_char(text_for_encode)))
print("Зашифрованный отрывок текста:\n", text_encoded)

Зашифрованный отрывок текста:
 ряьзрщнртддтагелнгльогылогтдакзлвакддпютсёюкгвзлвцеалёлуяизжрцнудсшииквнцложнфльлёчмзлэдхнетласэрвгфзж шгтещьзушагроляёзнфхвдъщаз юатщвцпоуа рзшчшршцлвгекьзйгсэъеенклекфлхмтцмовапявгозвкютцдмтхмьяпызлямянабзжхирвцбвхнф шлесмюасъюхьпимецзсйзехлёвадьоввбудйаэсудушзстцюрьзнффрчуылуольещйрухянхндуетркгнвхчсимяшэрплнгтэзгюлюаеаляьэзглаплцоекппзжжжуатъпгнщрвзчпалыженртддтьзнррщхмдънфехтдрщнфуавхлёмттцваолялелкцтщыпзгзални ексгвцкцзгсънхбхнхйгрвдьнхйрюаушрясжвхнвюкгввсрпекьзль шегоолыжейззсжуппэсхмзшчшршцлвгекьзнфьмжжй екьзль шянютйгхмвсласэыпвюелнгльвсюаеа тялплюрйгнфылрмеяютнрряиштъщаэдпясълбватдкдьэтциооаргсъжчдфялсъя ув мнфжжльюраюнрзазсяииксъряогрхгдеквлсэоуклушрядрыхгдеквлюаушишецлявжцийгрво ппексгвцкцзгкцоувяыпек зыбыло екухлячуыл шящнхьаямюкцуватдрсхмтщямцгыбкцелнгльвсыпогяшшцдрдььзлявжрврщушбътцоощаюрушжэднсълесмсъувцпожнфишэдисши зожнхтдйгямзаювнщпюширчылыштцыс вляьэряйгцквцгуегшцишаэрщонвссэыпвюкцэдисшиухишецляьэлесмвсцдсыюкгвсгватдхижжооюр зттвтзгжеу

In [23]:
text_encoded_order = Counter(get_ngrams(text_encoded,2)).most_common()
print("Частоты зашифрованного отрывка текста (первые 100):\n", text_encoded_order[:100])

Частоты зашифрованного отрывка текста (первые 100):
 [('ль', 33), ('ьз', 32), ('ек', 28), ('нф', 25), (' ш', 25), ('уш', 25), ('сэ', 21), ('хм', 21), ('съ', 21), ('тц', 20), ('вц', 19), ('уд', 19), ('рв', 19), ('нх', 19), ('тд', 18), ('ва', 18), ('дь', 17), ('вс', 17), ('рщ', 16), ('лё', 16), ('йг', 16), ('юр', 16), ('иш', 16), ('ря', 15), ('зл', 15), ('ши', 15), ('юа', 15), ('ям', 15), ('др', 15), (' з', 15), ('гв', 14), ('цл', 14), ('ля', 14), ('зн', 14), ('зг', 14), ('кц', 14), ('нр', 13), ('лн', 13), ('шр', 13), ('шц', 13), ('ле', 13), ('ех', 13), ('ыл', 12), ('ло', 12), ('пл', 12), ('хн', 11), ('ол', 11), ('кь', 11), ('ен', 11), ('пя', 11), ('цд', 11), ('же', 11), ('ящ', 11), ('ел', 10), ('нг', 10), ('ог', 10), ('юк', 10), ('гр', 10), ('ат', 10), ('зс', 10), ('хл', 10), ('уо', 10), ('ух', 10), ('ып', 10), ('вю', 10), ('ьо', 9), ('ют', 9), ('ал', 9), ('лу', 9), ('вг', 9), ('цз', 9), ('жж', 9), ('рх', 9), ('що', 9), ('еа', 8), ('зж', 8), ('ож', 8), ('эд', 8), ('хв', 8), ('яш', 8), (

In [24]:
print(f"Количество биграмм в исходном тексте (произведении): {len(RU_TOKKENS_BI)}")
print(f"Количество биграмм в отрывке текста: {len(text_encoded_order)}")

Количество биграмм в исходном тексте (произведении): 853
Количество биграмм в отрывке текста: 769


  - Количество биграмм в исходном тексте (произведении) и на отрывке текста не совпадают. Алгоритм расшифровки частотным методом для биграмм неоднозначен. 
  - Для дальнейшей расшифвровки возьмем наиболее часто встречающиеся биграммы исходного текста в количестве равному биграммам из отрывка текста

In [25]:
dict_decoder_from_text = dict(zip(map(lambda x: x[0], text_encoded_order), 
                                  map(lambda x: x[0], ru_char_order[:len(text_encoded_order)])))
text_decoded = "".join(map(lambda x: dict_decoder_from_text[x], get_pair_char(text_encoded)))
print("Расшифрованный отрывок текста:\n", text_decoded)

Расшифрованный отрывок текста:
 каа нороь г музан о талотаь ц  т кашият тлскра тя вигоатспдаезушобовмадр д зе о горл тазелтчик внаидда сутока  нмуужентпе чтытднуснився рнрутюекязян дети а по вглв апи тв п озе к меточует лиыл пабша тл буючдамонавяойе  соскунитоей фршд омар чго кнез лл иуп а и ном окоа е ятурлоано окоек буел хтчйсжаойюбрш уополн жчво рнивиенс воиколбыи сьдаы рууб ю жнатяртжевароь г а роно пыте  чь ное руойгоыл о керряза бвсривомеу бяи мия  бвотостхостпонанестоени нкачнойныскраалжии а о  с эсежевааромннсь а пекязян дети а е ахы мги а о  сбут по палик вриогзан о алнивичьряолкопое лоиныкт рокаонубдназ мтоех кь ашс  опеипрмтогчьзрятоуатиыше ы о коыйромеомспматокатавеиги ев вамап нкай вмиги евни нонд ентсеспоназьсьи мия  бво бамхурии лактлозьи к енурло срестьял скще кь фи пвсл ошкт бзан о алрита упрй неа ентснано нсю оседнко ньсэттооскутотищи зе оназзжовла зсть пол меиш жичоврслоял отднгенс капо гя уч эпронывноитал вриог баззжовк онд енс оскуаллисчскрами кь моы секолаецгквова

In [26]:
print(f"Точность дешифровки для отрывка текста: {text_accuracy(text_for_encode, text_decoded) * 100:.2f}%")

Точность дешифровки для отрывка текста: 6.42%


### Шифровка и расшифровка текста на **английском** языке частотным методом (для биграмм)

In [27]:
EN_TOKKENS_BI = list(map(lambda x: x[0], en_char_order))
EN_TOKKENS_BI_SHUFFLE = EN_TOKKENS_BI.copy()
random.shuffle(EN_TOKKENS_BI_SHUFFLE)
dict_encoder_en = dict(zip(EN_TOKKENS_BI, EN_TOKKENS_BI_SHUFFLE))
print("Токкены биграмм для английского языка (первые 100):\n", EN_TOKKENS_BI[:100])

Токкены биграмм для английского языка (первые 100):
 ['e ', ' t', 'd ', 'he', 'th', ' a', 's ', 't ', ' h', 'in', 'an', 'er', ' s', ' w', 'n ', 're', 'nd', ' o', 'y ', 'r ', 'ed', 'at', 'on', ' i', 'ha', 'o ', 'en', 'hi', ' b', 'ng', 'to', 'ou', 'g ', 'is', 'it', ' f', 'as', 'es', ' c', 'or', ' m', 'f ', ' p', 'te', 'ar', 'st', 'se', ' d', 'nt', 'le', 'of', 'h ', 'me', 'a ', ' r', ' n', 'ne', 'ro', 've', 'al', 'ti', 'de', 'ho', 'ri', 'ea', 'll', ' l', 'ce', 'l ', 'co', 'no', 'm ', 'wa', 'wh', ' e', 'be', 'ad', 'om', 'ut', 'ch', 'sh', 'el', 'wi', 'ot', 'ly', 'im', 'ow', ' g', 'ss', 'li', 'ma', 'si', 'nc', 'so', 'us', 'ta', 'un', 'ai', 'id', 'rs']


In [28]:
text_for_encode = text_war_en[1030:4322]
print("Отрывок текста для шифровки:\n", text_for_encode)

Отрывок текста для шифровки:
 i will have nothing more to do with you and you are no longer my friend no longer my faithful slave as you call yourself but how do you do see have frightened yousit down and tell me all the news it was in july and the speaker was the wellknown anna pavlovna scherer maid of honor and favorite of the empress marya fedorovna with these words she greeted prince vasili kuragin a man of high rank and importance who was the first to arrive at her reception anna pavlovna had had a cough for some days she was as she said suffering from la grippe grippe being then a new word in st petersburg used only by the elite all her invitations without exception written in french and delivered by a scarletliveried footman that morning ran as follows if you have nothing better to do count or prince and if the prospect of spending an evening with a poor invalid is not too terrible shall be very charmed to see you tonight between and annette scherer heavens what a virulent attac

In [29]:
text_encoded = "".join(map(lambda x: dict_encoder_en[x], get_pair_char(text_for_encode)))
print("Зашифрованный отрывок текста:\n", text_encoded)

Зашифрованный отрывок текста:
 necsyvfyhmorlkpewktxgdczkfdzsd jut nfdhcgaoffdhcpiorlkragovvxqzmdhdwhhoflkragovvxqzmdh tpez gfcqhmorlfwntnhelegffddmprxcqulwfyasfsdzfdhcsdmluffyhmorakvaoghhomwntn fzssdbtelxhkfysgfaaelyvkfmfvbejgiut jlftfxletayelxhkfmfmlujbcax jlfkfmf jyscllkbtel xrnmyzuicscmlsraxaxkk tofinfygokhelxhdhhmkhutorinkfmfthbaczeckkpictdhomkhicsc jut npeklormsx gitloroyufbsofyiwkhu glfhaner xatrxlrnuyxlinfyva nxadjelxhtfbakhwilsorrldzssgipeoryotpzsbyel yl orktfyaxmajouetagoel xrnmyzuicscfyplfyplelhetnbedhkhmlemordswemlmf jlfelgitlorglhrml dyrdwf dhryri crpdwtkoroyudujquvef kfmfxlrnyfeamsx tfxlpny eiaxvsdmtxeqomvlxougaqkfmfth qbselyvfyaxtfznutktuipa juttjlwthyyuetago jdwfkhhtfxlakhhsrelxhfsysl axomquugrnkppiehzvl axhkofpooouyxlpektkkkhlmf magaelgipoyvasgikewntnfyhmorlkpewktxnxfkaxkfdzsdhetnc vlxqyiwkhuelxhtf speoryikgujdcvl sothhitf elxlrrhhwktxcspeely olxqwko  qofuzvbookfolkfaxdwxsortllegfnx gaxugsrpiaaofbymlufwntnkfgovaogqueimuhhelxhel xeibsmlsraxaxfyvormpa jgrzsrnwsgmehc elfkb

In [30]:
text_encoded_order = Counter(get_ngrams(text_encoded,2)).most_common()
print("Частоты зашифрованного отрывка текста (первые 100):\n", text_encoded_order[:100])

Частоты зашифрованного отрывка текста (первые 100):
 [('or', 71), ('of', 49), ('el', 45), ('fy', 41), ('pe', 35), ('zs', 35), ('kf', 33), ('mf', 32), ('xl', 30), ('ax', 30), ('kt', 29), ('gi', 28), ('hh', 26), ('ml', 26), ('tf', 25), (' j', 24), ('wk', 23), ('go', 23), ('xh', 23), ('f ', 22), ('ga', 21), ('he', 21), ('lx', 21), ('rn', 20), ('fm', 19), ('kh', 19), ('ao', 18), ('tn', 18), ('qu', 18), ('om', 18), ('cz', 17), ('dh', 17), ('fx', 17), ('l ', 17), ('rr', 17), ('xq', 16), ('hk', 16), ('lk', 15), ('ut', 15), ('nx', 15), ('oo', 15), ('sp', 15), ('fd', 14), ('lf', 14), ('le', 14), ('ya', 14), ('sr', 14), ('ek', 14), ('ro', 14), ('gr', 14), ('tp', 13), ('sg', 13), ('ls', 13), ('in', 13), ('so', 13), ('lg', 13), ('ug', 13), ('yv', 12), ('rl', 12), ('tx', 12), ('dz', 12), ('dw', 12), ('hu', 12), ('kl', 12), ('rm', 12), ('eo', 12), ('yf', 12), ('pn', 12), ('si', 12), ('oh', 12), ('ra', 11), ('nt', 11), ('ys', 11), ('sc', 11), ('by', 11), ('rp', 11), ('y ', 11), (' q', 11), ('fg', 11)

In [31]:
print(f"Количество биграмм в исходном тексте (произведении): {len(EN_TOKKENS_BI)}")
print(f"Количество биграмм в отрывке текста: {len(text_encoded_order)}")

Количество биграмм в исходном тексте (произведении): 600
Количество биграмм в отрывке текста: 581


  - Количество биграмм в исходном тексте (произведении) и на отрывке текста не совпадают. Алгоритм расшифровки частотным методом для биграмм неоднозначен. 
  - Для дальнейшей расшифвровки возьмем наиболее часто встречающиеся биграммы исходного текста в количестве равному биграммам из отрывка текста

In [32]:
dict_decoder_from_text = dict(zip(map(lambda x: x[0], text_encoded_order), 
                                  map(lambda x: x[0], en_char_order[:len(text_encoded_order)])))
text_decoded = "".join(map(lambda x: dict_decoder_from_text[x], get_pair_char(text_encoded)))
print("Расшифрованный отрывок текста:\n", text_decoded)

Расшифрованный отрывок текста:
 etshroheele esthndaleutos tiayre cct poted t potrse esno oxe fimoude s tesno oxe fimoulathmbpepsele teiohiatarpe poonuxi bsuhek owti potay wieheele emuceb sngiohicl aayubd y s wapesad ros t ldzier creten  h gobd y s t  wosdqinretes t rewatpesubd ap i yfutswh wseininlola ta he oo d y ouelo  ce a s t w fftoevlorsksoungo tswhre cctthrie panseryoe fiieee tu ndhop tecketylmowy h iey ha heucctmoykd y n ffo zemee vetibterthe maof a ed mcise anheinfoncryov od ap i yfutswhheagheagd athiodouo  w je rveo wt reted eryoe rtfr wabkeder ouurullsbedetae fizoos bvsr s t  h i livpansn  hceadepiniqooalc ngprtrnewws t w omeed roheinn yc cancaokre cvosuw ymryov orederu sn  hem ssed y owwaisinng bne ihtrswokaisinas tqu mey hthanloo amr foedd erqurok erttiohiheele esthndalorruins tiayathilipr fu ndhod y n kithe u  qospiprkisi snir d  hit sndalshthd adrp fndnkom tdild ms rps indeybe yoarpeorp inneserssa t e wieiohis  ouceb bepwn sd y d apepee wseininheaieaokrele a ibstvwolid ru

In [33]:
print(f"Точность дешифровки для отрывка текста: {text_accuracy(text_for_encode, text_decoded) * 100:.2f}%")

Точность дешифровки для отрывка текста: 11.97%


### Выводы
  - Частотный подход на биграммах расшифровывает хуже чем для одиночных символов. Для биграмм требуется еще большая длина зашифрованного отрывка текста

## Часть 3.

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

    - предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
    - реализуйте и протестируйте его, убедитесь, что результаты улучшились.

**Алгоритм решения**
    
    - Для решения задачи на известном тексте предварительно расчитаем частоты и далее вероятности перехода в тексте от символа к символу (статистика биграмм). 
    - Полученную матрицу вероятностей последовательности символов будем использовать для оценки вероятности зашифрованного текста (токкены должны быть одинаковые).
    - Далее с помощью алгоритма Метрополиса-Гастингса будем семплировать перестановки произвольных пар символов и оценивать вероятность нового зашифрованного текста. С учетом предыдущей оценки будем принимать или отклонять перестановку и постепенно  расшифровывать текст

In [206]:
class Decoder:
    EN_TOKKENS = list('abcdefghijklmnopqrstuvwxyz ')
    RU_TOKKENS = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')
    
    def __init__(self, text, lang_ru=True):
        if lang_ru:
            self.chars = Decoder.RU_TOKKENS.copy()
        else:
            self.chars = Decoder.EN_TOKKENS.copy()        
        
        self.chars_len = len(self.chars)        
        self.char_index = dict(zip(self.chars, range(self.chars_len)))
        
        #расчет частот матрицы переходов от символа к символу
        self.pm = np.zeros((self.chars_len, self.chars_len))     
        for text_idx in range(1, len(text)):
            idx_cur = self.char_index[text[text_idx - 1]]
            idx_cur_next = self.char_index[text[text_idx]]
            self.pm[idx_cur, idx_cur_next] += 1           
        self.pm = np.clip(self.pm, a_min=1, a_max=None)       
        self.pm = self.pm / self.pm.sum(axis=1)[:, np.newaxis] #нормировка
        self.pm_log = np.log(self.pm) #предварительное логарифмирование для ускорения счета (произв. вероятностей)
    
    def text_score_log(self, text, shuffled_tokkens):
        text = self.decoder(text, shuffled_tokkens)        
        score_log = 0
        for text_idx in range(1, len(text)):
            idx_cur = self.char_index[text[text_idx - 1]]
            idx_cur_next = self.char_index[text[text_idx]]
            score_log += self.pm_log[idx_cur, idx_cur_next]
        return score_log
          
    def decoder(self, text, tokkens):
        dict_decoder = dict(zip(tokkens, self.chars))
        return "".join(map(lambda x: dict_decoder[x], text))
        

    def random_idx(self, max_value_included):
        idx1 = idx2 = 0
        while idx1 == idx2:
            idx1 = random.randint(0, max_value_included)
            idx2 = random.randint(0, max_value_included)
        return idx1, idx2
 
    def swap_tokkens(self, tokkens, idx1, idx2):
        tokkens[idx1], tokkens[idx2] = tokkens[idx2], tokkens[idx1]
        return tokkens
    
    def decode(self, text, num_iteration): 
        shuffled_tokkens = self.chars.copy()
        random.shuffle(shuffled_tokkens)
        self.decode_tokkens = shuffled_tokkens.copy()       
        max_score_log = score_log = self.text_score_log(text, shuffled_tokkens)
        
        for i in range(num_iteration):
            idx1, idx2 = self.random_idx(self.chars_len - 1)
            shuffled_tokkens = self.swap_tokkens(shuffled_tokkens, idx1, idx2)  
            
            new_score_log = self.text_score_log(text, shuffled_tokkens)             
            if new_score_log <= score_log:
                if np.log(random.uniform(0,1)) > new_score_log - score_log:
                    shuffled_tokkens = self.swap_tokkens(shuffled_tokkens, idx1, idx2)                    
                else:
                    score_log = new_score_log         
            else:
                score_log = new_score_log
                if new_score_log > max_score_log:
                    max_score_log = new_score_log
                    self.decode_tokkens = shuffled_tokkens.copy()
        
        return self.decoder(text, self.decode_tokkens)

### Шифровка и расшифровка текста на русском языке основанный на MCMC-сэмплировании

In [207]:
text_for_encode = text_anna_ru[1001:3959]
print("Отрывок текста для шифровки:\n", text_for_encode)

Отрывок текста для шифровки:
 в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения время подтвердило эту высокую оценку из статей и книг на всех языках мира посвященных анне карениной можно составить целую библиотеку я без колебаний назвал анну каренину величайшим социальным романом во всей мировой литературе писал томас манн значение романа толстого состоит не в эстетической ценности отдельных картин а в художественной завершенности целого войну и мир толстой называл книгой о прошлом в начале года он просил редактора журнала русский вестник м н каткова в оглавлении и даже в объявлении не называть его сочинение романом для меня это очень важно и потому очень прошу вас об этом толстой мог бы обосновать свое определение жанра книга ссылкой на гегеля которого он внимательно перечитывал в годы работы над войной и миром гегель называл книгой эпические произведения связанные 

In [208]:
RU_TOKKENS_SHUFFLE = RU_TOKKENS.copy()
random.shuffle(RU_TOKKENS_SHUFFLE)
dict_encoder_ru = dict(zip(RU_TOKKENS, RU_TOKKENS_SHUFFLE))
text_encoded = "".join(map(lambda x: dict_encoder_ru[x], text_for_encode))
print("Зашифрованный отрывок текста:\n", text_encoded)

Зашифрованный отрывок текста:
 жёнэжэяёюэяунбёзэдвзэсэёэсюэянъфёгвоьэдэсойбвпъфёюумюукэзпъёхъщоёйбдэжбйбвпэаёвзюущнъфёсдъконъёоёводъёоёсдужнэбёнбкчжудчаёхэвбдбёъёнувёюбудомяёьъхэтбвзжбннэсэёомэкюутбнолёжюбялёгэхзжбюходэёизъёжчвэпъфёэыбнпъёомёвзузбаёоёпносёнуёжвбьёлмчпуьёяоюуёгэвжл бннчьёуннбёпуюбнонэаёяэтнэёвэвзужозцёыбдъфёкокдоэзбпъёлёкбмёпэдбкуноаёнумжудёуннъёпуюбнонъёжбдойуащояёвэыоудцнчяёюэяунэяёжэёжвбаёяоюэжэаёдозбюузъюбёговудёзэяувёяуннёмнуйбнобёюэяунуёзэдвзэсэёвэвзэозёнбёжёивзбзойбвпэаёыбннэвзоёэзхбдцнчьёпуюзонёуёжёьъхэтбвзжбннэаёмужбющбннэвзоёыбдэсэёжэанъёоёяоюёзэдвзэаёнумчжудёпносэаёэёгюэщдэяёжёнуйудбёсэхуёэнёгюэводёюбхупзэюуётъюнудуёюъввпоаёжбвзнопёяёнёпузпэжуёжёэсдуждбнооёоёхутбёжёэкелждбнооёнбёнумчжузцёбсэёвэйонбнобёюэяунэяёхдлёябнлёизэёэйбнцёжутнэёоёгэзэяъёэйбнцёгюэщъёжувёэкёизэяёзэдвзэаёяэсёкчёэкэвнэжузцёвжэбёэгюбхбдбнобётунюуёпносуёввчдпэаёнуёсбсбдлёпэзэюэсэёэнёжнояузбдцнэёгбюбйозчжудёжёсэхчёюукэзчёнухёжэанэаёоёяоюэяёсбсбдцёнумчжудёпносэаёигойбвпобёгюэомжбхбнолёвжлмуннчб

In [209]:
%%time
dec_ru = Decoder(text_anna_ru + text_war_ru)
decoded_text = dec_ru.decode(text_encoded, num_iteration=50000)
print("Расшифрованный отрывок текста:\n", decoded_text)

Расшифрованный отрывок текста:
 в новом романе толстого огромную психологическую разработку души человеческой страшную глубину и силу и главное небывалый доселе у нас реализм художественного изображения время подтвердило эту высокую офенку из статей и книг на всех языках мира посвященных анне карениной можно составить фелую библиотеку я без колебаний назвал анну каренину величайшим софиальным романом во всей мировой литературе писал томас манн значение романа толстого состоит не в эстетической фенности отдельных картин а в художественной завершенности фелого войну и мир толстой называл книгой о прошлом в начале года он просил редактора журнала русский вестник м н каткова в оглавлении и даже в объявлении не называть его сочинение романом для меня это очень важно и потому очень прошу вас об этом толстой мог бы обосновать свое определение жанра книга ссылкой на гегеля которого он внимательно перечитывал в годы работы над войной и миром гегель называл книгой эпические произведения связанны

In [210]:
print(f"Точность дешифровки для отрывка текста: {text_accuracy(text_for_encode, decoded_text) * 100:.2f}%")

Точность дешифровки для отрывка текста: 99.43%


### Шифровка и расшифровка текста на английском языке основанный на MCMC-сэмплировании

In [211]:
text_for_encode = text_war_en[1030:4322]
print("Отрывок текста для шифровки:\n", text_for_encode)

Отрывок текста для шифровки:
 i will have nothing more to do with you and you are no longer my friend no longer my faithful slave as you call yourself but how do you do see have frightened yousit down and tell me all the news it was in july and the speaker was the wellknown anna pavlovna scherer maid of honor and favorite of the empress marya fedorovna with these words she greeted prince vasili kuragin a man of high rank and importance who was the first to arrive at her reception anna pavlovna had had a cough for some days she was as she said suffering from la grippe grippe being then a new word in st petersburg used only by the elite all her invitations without exception written in french and delivered by a scarletliveried footman that morning ran as follows if you have nothing better to do count or prince and if the prospect of spending an evening with a poor invalid is not too terrible shall be very charmed to see you tonight between and annette scherer heavens what a virulent attac

In [212]:
EN_TOKKENS_SHUFFLE = EN_TOKKENS.copy()
random.shuffle(EN_TOKKENS_SHUFFLE)
dict_encoder_en = dict(zip(EN_TOKKENS, EN_TOKKENS_SHUFFLE))
text_encoded = "".join(map(lambda x: dict_encoder_en[x], text_for_encode))
print("Зашифрованный отрывок текста:\n", text_encoded)

Зашифрованный отрывок текста:
 zr zyyrgoumrhsagzhdrlsqmrasrcsr zagrksirohcrksiroqmrhsryshdmqrlkrbqzmhcrhsryshdmqrlkrbozagbiyrjyoumrojrksireoyyrksiqjmybrfiargs rcsrksircsrjmmrgoumrbqzdgamhmcrksijzarcs hrohcramyyrlmroyyragmrhm jrzar ojrzhrtiykrohcragmrjxmopmqr ojragmr myyphs hrohhorxouysuhorjegmqmqrlozcrsbrgshsqrohcrbousqzamrsbragmrmlxqmjjrloqkorbmcsqsuhor zagragmjmr sqcjrjgmrdqmmamcrxqzhemruojzyzrpiqodzhrorlohrsbrgzdgrqohprohcrzlxsqaohemr gsr ojragmrbzqjarasroqqzumroargmqrqmemxazshrohhorxouysuhorgocrgocroresidgrbsqrjslmrcokjrjgmr ojrojrjgmrjozcrjibbmqzhdrbqslryordqzxxmrdqzxxmrfmzhdragmhrorhm r sqcrzhrjarxmamqjfiqdrijmcrshykrfkragmrmyzamroyyrgmqrzhuzaoazshjr zagsiarmnemxazshr qzaamhrzhrbqmhegrohcrcmyzumqmcrfkrorjeoqymayzumqzmcrbssalohragoarlsqhzhdrqohrojrbsyys jrzbrksirgoumrhsagzhdrfmaamqrasrcsresiharsqrxqzhemrohcrzbragmrxqsjxmearsbrjxmhczhdrohrmumhzhdr zagrorxssqrzhuoyzcrzjrhsarassramqqzfymrjgoyyrfmrumqkregoqlmcrasrjmmrksirashzdgarfma mmhrohcrohhmaamrjegmqmqrgmoumhjr goaroruzqiymharoaao

In [213]:
%%time
dec_en = Decoder(text_war_en, lang_ru=False)
decoded_text = dec_en.decode(text_encoded, num_iteration=20000)
print("Расшифрованный отрывок текста:\n", decoded_text)

Расшифрованный отрывок текста:
 i will have nothing more to do with you and you are no longer my friend no longer my faithful slave as you call yourself but how do you do see have frightened yousit down and tell me all the news it was in quly and the speaker was the wellknown anna pavlovna scherer maid of honor and favorite of the empress marya fedorovna with these words she greeted prince vasili kuragin a man of high rank and importance who was the first to arrive at her reception anna pavlovna had had a cough for some days she was as she said suffering from la grippe grippe being then a new word in st petersburg used only by the elite all her invitations without exception written in french and delivered by a scarletliveried footman that morning ran as follows if you have nothing better to do count or prince and if the prospect of spending an evening with a poor invalid is not too terrible shall be very charmed to see you tonight between and annette scherer heavens what a virulent att

In [214]:
print(f"Точность дешифровки для отрывка текста: {text_accuracy(text_for_encode, decoded_text) * 100:.2f}%")

Точность дешифровки для отрывка текста: 99.94%


### Выводы
   - Подход основанный на MCMC-сэмплировании показал наилуший результат, точность для русского текста 99,4%, а для английского 99.9%  
   - Текст расшифрован не на полные 100% корректно, однако, он читаем и смысл понятен. 
   
   - Текст если необходимо уже можно дорасшифровать вручную, возможно увелечение количества итерациq может улучшить результат 

## Часть 4.

Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹
⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠
⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშ
ჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹ
სდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

In [215]:
text1 = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
text2 = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

print(f"Количество токкенов английского языка: {len(EN_TOKKENS)}")
print(f"Количество токкенов русского языка: {len(RU_TOKKENS)}")
print(f"Количество токкенов зашифрованных отрывков текста: {len(set(text1))} и {len(set(text2))}")

Количество токкенов английского языка: 27
Количество токкенов русского языка: 34
Количество токкенов зашифрованных отрывков текста: 28 и 28


   - Количество токкенов зашифрованных отрывков текста больше чем число токкенов английского языка. Поэтому будем расшифровывать на русском языке

In [235]:
%%time
change_tokkens1 = dict(zip(set(text1), RU_TOKKENS[:len(set(text1))]))
text_encoded1 = "".join(map(lambda x: change_tokkens1[x], text1))
decoded_text = dec_ru.decode(text_encoded1, num_iteration=60000)
print("Расшифрованный отрывок текста:\n", decoded_text)

Расшифрованный отрывок текста:
 если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
Wall time: 12 s


In [233]:
%%time
change_tokkens2 = dict(zip(set(text2), RU_TOKKENS[:len(set(text2))]))
text_encoded2 = "".join(map(lambda x: change_tokkens2[x], text2))
decoded_text = dec_ru.decode(text_encoded2, num_iteration=60000)
print("Расшифрованный отрывок текста:\n", decoded_text)

Расшифрованный отрывок текста:
 если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
Wall time: 13.3 s


### Выводы
   - Подход основанный на MCMC-сэмплировании позволил расшировать текст.
   
   - Текст расшифрован не полностью корректно, однако, он читаем и смысл понятен. 

## Часть 6.	Бонус

Какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

### Предожения по области применения модели
  - В рекомендательных системах. 
        - На основе анализа совместных покупок, возможно сформировать матрицы вероятностей совместных покупок (различных n-gram),  понятно, что последовательность продуктов в чеке не должна влиять на оценку вероятности и матрица получиться симметричная. 
        - При формировании корзины покупателем можно на лету оценивать при добавлении какого продукта в корзину получается максимальная оценка вероятности корзины и рекомендовать данный товар.
        - Либо еще можно рекомендовать замену продукта на аналогичный (например, памперс другой фирмы) если вероятность корзины и прибыль магазина при этом увеличивается.
  - Предсказание поломок элементов оборудования. 
        - При достаточной статистике последовательности событий/сообщений/поломок/нормальной_работы оборудования возможно построить матрицу частот/вероятностей переходов между событиями. 
        - Добавляя в текущию последовательность событий оборудования следующее возможное оценивается совокупная вероятность. Перебрав все возможные продолжения событий можно найти наиболее вероятные.
  - Формирование личных рекомендаций фитнес треккера. Основной целью рекомендаций фитнес трекера будет поддержание текущего веса (основная цель настроиться на индивидуальные характеристики организма). 
        - Собрать в единую последовательность ежедневные события показаний фитнес трекера (кол-во шагов), оценку калорийности потребляемой пищи и замер веса.
        - Далее необходимо уменьшить количество возможных значений показаталей: например, значения калорий привести к шкале с шагом 100ккал, количество шагов к шкале с 500 шагами, вес имеет три значения: упал, вырос, тот же по сравнению с пред. днем.
        - Далее накапливаем статистику и настраиваем матрицу вероятности переходов между состояниями
        - Далее, например, пользователь знает, что сегодня он ходить много не будет (вводит объем шагов), тогда трекер может порекомендовать объем калорий который можно съесть при котором наиболее вероятно вес не измениться (Для этого оценивается вероятность для всех возможных объемов калории)
